In [19]:
from mbientlab.metawear import MetaWear, libmetawear
from mbientlab.metawear.cbindings import *
from ctypes import c_void_p, POINTER, CFUNCTYPE
import time

In [20]:
# MAC address of the MetaWear device
device_address = 'D4:06:BD:85:8F:23'

# Create and connect the device
device = MetaWear(device_address)
device.connect()

In [21]:
data_list = []

# Callback function to handle incoming data
def data_handler(context, data):
    # Convert the data pointer to a usable format, assuming it's CartesianFloat for demonstration
    if data.contents.type_id == DataTypeId.CARTESIAN_FLOAT:
        value = cast(data.contents.value, POINTER(CartesianFloat)).contents
        timestamp = int(data.contents.epoch)
        sensor = context.decode('utf-8')
        print("Sensor: %.2f Timestamp: %.2f X: %.2f Y: %.2f Z: %.2f" % (sensor, timestamp, value.x, value.y, value.z))
        data_list.append([timestamp, value.x, value.y, value.z])
        
        


In [22]:

# Define the callback type
FnVoid_VoidP_DataP = CFUNCTYPE(None, c_void_p, POINTER(Data))

# Setup callback
callback = FnVoid_VoidP_DataP(data_handler)

# Configure accelerometer
libmetawear.mbl_mw_acc_set_odr(device.board, 100.0)  # Set output data rate to 100 Hz
libmetawear.mbl_mw_acc_set_range(device.board, 16.0)  # Set range to +/-16g
libmetawear.mbl_mw_acc_write_acceleration_config(device.board)

# Get the accelerometer data signal
acc_signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)



libmetawear.mbl_mw_gyro_bmi160_write_config(device.board)
gyro_signal = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(device.board)



# Subscribe to the accelerometer data signal
libmetawear.mbl_mw_datasignal_subscribe(acc_signal, None, callback, "acc".encode('utf-8'))
libmetawear.mbl_mw_datasignal_subscribe(gyro_signal, None, callback, "gyro".encode('utf-8'))


# Enable accelerometer
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_acc_start(device.board)
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(device.board)
libmetawear.mbl_mw_gyro_bmi160_start(device.board)

# Stream data for a duration (e.g., 30 seconds)
time.sleep(5)

# Stop and clean up
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_datasignal_unsubscribe(acc_signal)
libmetawear.mbl_mw_debug_disconnect(device.board)
print("Disconnected and cleaned up")

Exception ignored on calling ctypes callback function: <function data_handler at 0x000002811939D580>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Temp\ipykernel_26424\3904960911.py", line 9, in data_handler
AttributeError: 'NoneType' object has no attribute 'decode'
Exception ignored on calling ctypes callback function: <function data_handler at 0x000002811939D580>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Temp\ipykernel_26424\3904960911.py", line 9, in data_handler
AttributeError: 'NoneType' object has no attribute 'decode'
Exception ignored on calling ctypes callback function: <function data_handler at 0x000002811939D580>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Temp\ipykernel_26424\3904960911.py", line 9, in data_handler
AttributeError: 'NoneType' object has no attribute 'decode'
Exception ignored on calling ctypes callback function: <function data_handler at 0x000002811939D580>
Traceback (most r

Disconnected and cleaned up


In [15]:
device.on_disconnect = lambda status: print ("we are disconnected!")
device.disconnect()

we are disconnected!


In [23]:
for i in range(100):
    print(data_list[i])

IndexError: list index out of range